<a href="https://colab.research.google.com/github/JPOrellana/LAB5_VEC/blob/main/Laboratorio_4_Vectorizacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NOMBRES: José Pablo

APELLIDOS: Orellana Orellana

CARNE: 21970

FECHA: 04 de septiembre del 2025

### Librerías

In [1]:
from collections import Counter
import math
import numpy as np
import pandas as pd

**Ejercicio 1**
Cree un corpus a su gusto como el visto en clase, cálcule PPMI, pero aplicando Lapace Smoothing.

In [3]:
# --- Corpus  ---
corpus = [
    "madrid es la capital de españa",
    "paris es la capital de francia",
    "roma es la capital de italia",
    "berlin es la capital de alemania",
    "lisboa es la capital de portugal",
    "el rey es un hombre",
    "la reina es una mujer",
    "un príncipe es hijo del rey",
    "una princesa es hija de la reina",
    "el doctor es un hombre",
    "la doctora es una mujer",
    "el actor es famoso",
    "la actriz es famosa",
    "madrid está en españa",
    "paris está en francia",
    "roma está en italia",
    "berlin está en alemania",
    "lisboa está en portugal",
    "hombre y hombres son formas relacionadas",
    "mujer y mujeres son formas relacionadas",
    "el rey y la reina gobiernan un reino",
    "hombre y mujer viven en ciudades como madrid paris roma",
]

def tokenize(s): 
    return s.lower().split()

tokenized = [tokenize(s) for s in corpus]

# --- Matriz de co-ocurrencias con ventana ±2 ---
window_size = 2
vocab = sorted({w for sent in tokenized for w in sent})
idx = {w:i for i, w in enumerate(vocab)}
V = len(vocab)

cooc = np.zeros((V, V), dtype=np.float64)
unigram = Counter()

for sent in tokenized:
    n = len(sent)
    for i, w in enumerate(sent):
        wi = idx[w]
        unigram[w] += 1
        L, R = max(0, i-window_size), min(n, i+window_size+1)
        for j in range(L, R):
            if j == i: 
                continue
            cj = idx[sent[j]]
            cooc[wi, cj] += 1.0

# --- Laplace smoothing + PPMI ---
alpha = 1.0
cooc_lap = cooc + alpha

total = cooc_lap.sum()
p_wc = cooc_lap / total
p_w = p_wc.sum(axis=1, keepdims=True)
p_c = p_wc.sum(axis=0, keepdims=True)

eps = 1e-12
PMI = np.log2((p_wc + eps) / (p_w @ p_c + eps))
PPMI = np.maximum(PMI, 0.0)

# Variables útiles para el Ejercicio 2
def get_vec(word:str) -> np.ndarray:
    return PPMI[idx[word]]


**Ejercicio 2**

Crear 10 analigias validas basadas en su texto, tome en cuenta que es necesario utilizar el codigo visto en clase y ajuste su corpus para poder lograr obtener todas las analogias

In [4]:
def cosine(a: np.ndarray, b: np.ndarray) -> float:
    na, nb = np.linalg.norm(a), np.linalg.norm(b)
    if na == 0 or nb == 0:
        return 0.0
    return float(np.dot(a, b) / (na * nb))

def most_similar(vec: np.ndarray, exclude=set(), topn=5):
    sims = []
    for w in vocab:
        if w in exclude: 
            continue
        sims.append((w, cosine(vec, get_vec(w))))
    sims.sort(key=lambda x: x[1], reverse=True)
    return sims[:topn]

def analogy(a: str, b: str, c: str, topn:int=5):
    vec = get_vec(b) - get_vec(a) + get_vec(c)
    return most_similar(vec, exclude={a,b,c}, topn=topn)

tests = [
    ("hombre", "rey", "mujer", "reina"),
    ("hombre", "doctor", "mujer", "doctora"),
    ("actor", "actriz", "famoso", "famosa"),
    ("francia", "paris", "españa", "madrid"),
    ("italia", "roma", "portugal", "lisboa"),
    ("alemania", "berlin", "italia", "roma"),
    ("rey", "reina", "hijo", "hija"),
    ("hombre", "hombres", "mujer", "mujeres"),
    ("capital", "madrid", "francia", "paris"),
    ("capital", "roma", "alemania", "berlin"),
]

print("Analogías (a:b :: c: ?)\n")
for a,b,c,expected in tests:
    try:
        preds = analogy(a,b,c, topn=5)
        top1 = preds[0] if preds else ("<sin pred>", 0.0)
        print(f"{a}:{b} :: {c}: ?  | esperado={expected:8s} | top1={top1[0]:8s} sim={top1[1]:.3f} | top5={[w for w,_ in preds]}")
    except KeyError as e:
        print(f"{a}:{b} :: {c}: ?  | palabra fuera de vocabulario: {e}")


Analogías (a:b :: c: ?)

hombre:rey :: mujer: ?  | esperado=reina    | top1=doctora  sim=0.432 | top5=['doctora', 'es', 'actor', 'princesa', 'del']
hombre:doctor :: mujer: ?  | esperado=doctora  | top1=doctora  sim=0.501 | top5=['doctora', 'princesa', 'es', 'actor', 'ciudades']
actor:actriz :: famoso: ?  | esperado=famosa   | top1=el       sim=0.267 | top5=['el', 'capital', 'de', 'famosa', 'doctora']
francia:paris :: españa: ?  | esperado=madrid   | top1=madrid   sim=0.682 | top5=['madrid', 'ciudades', 'roma', 'en', 'berlin']
italia:roma :: portugal: ?  | esperado=lisboa   | top1=como     sim=0.719 | top5=['como', 'berlin', 'lisboa', 'paris', 'madrid']
alemania:berlin :: italia: ?  | esperado=roma     | top1=lisboa   sim=1.000 | top5=['lisboa', 'españa', 'francia', 'portugal', 'roma']
rey:reina :: hijo: ?  | esperado=hija     | top1=doctora  sim=0.415 | top5=['doctora', 'un', 'capital', 'reino', 'el']
hombre:hombres :: mujer: ?  | esperado=mujeres  | top1=relacionadas sim=0.391 | top5=